In [120]:
import pandas as pd
data = pd.read_csv('Mall_Customers.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   CustomerID              200 non-null    int64 
 1   Gender                  200 non-null    object
 2   Age                     200 non-null    int64 
 3   Annual Income (k$)      200 non-null    int64 
 4   Spending Score (1-100)  200 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 7.9+ KB


In [121]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np

In [122]:
x = pd.DataFrame(data)

# Converter a coluna 'Gender' para string, remover espaços em branco e converter para minúsculas
x['Gender'] = x['Gender'].astype(str).str.strip().str.lower()

# Codificar a coluna 'Gender'
x['Gender'] = x['Gender'].map({'male': 0, 'female': 1})

# Verificar se o mapeamento funcionou corretamente
x['Gender']

0      0
1      0
2      1
3      1
4      1
      ..
195    1
196    1
197    0
198    0
199    0
Name: Gender, Length: 200, dtype: int64

In [123]:
# Separar as características (features) e os rótulos (labels)
y = (data['Spending Score (1-100)'] > 50).astype(int).values
# Os maiores que 50 terão 1 e os menores terão 0 como valor


limite = 70
# Função para discretizar os valores
def discretizar(valor):
    if valor <= limite:
        return 0
    else:
        return 1

# Aplicar a função à coluna 'valores' usando o método apply
x['Annual Income (k$)_discretizados'] = x['Annual Income (k$)'].apply(discretizar)
limite = 28
x['Age_discretizados'] = x['Age'].apply(discretizar)

x = x.drop(['Annual Income (k$)', 'Age', 'Spending Score (1-100)', 'CustomerID'], axis=1)


In [124]:
x

,Gender,Annual Income (k$)_discretizados,Age_discretizados
0,0,0,0
1,0,0,0
2,1,0,0
3,1,0,0
4,1,0,1
...,...,...,...
195,1,1,1
196,1,1,1
197,0,1,1
198,0,1,1


In [125]:
y = pd.DataFrame(y)
y

,0
0,0
1,1
2,0
3,1
4,0
...,...
195,1
196,0
197,1
198,0


In [128]:
# Dividindo treino e teste com sklearn

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

# test size: Proporção ou número absoluto de amostras que irão compor o conjunto de teste. Pode ser um float entre 0.0 e 1.0.
# Random state: Define a semente usada pelo gerador de números aleatórios. Isso é útil para garantir que a divisão dos dados seja reprodutível. 
#Um valor inteiro fixa a semente, permitindo que você obtenha os mesmos conjuntos de treino e teste em execuções diferentes.

In [137]:
import tensorflow as tf

tf.random.set_seed(2)
#Isso é importante para garantir que os resultados das operações que envolvem aleatoriedade sejam reprodutíveis. 
#Quando você define uma semente aleatória, qualquer operação aleatória que você executar 
#(como inicialização de pesos, embaralhamento de dados, ou divisão de dados em lotes) produzirá os mesmos resultados a cada vez que o código for executado.



# CRIANDO A ESTRUTURA DA REDE NEURAL SEQUENCIAL com tensor float
# As camadas densas são completamente conectadas e são frequentemente usadas em redes neurais feedforward.
# Existem as camadas convolucionais, usadas principalmente em redes neurais convolucionais (CNNs) para tarefas de visão computacional.
# Existem as camadas recorrentes, usadas em redes neurais recorrentes (RNNs) para processamento de sequências, como em NLP.
# Ainda existem outros tipos de camada, dependendo da necessidade.

# CAMADA DE ENTRADA
#deve ser passada uma tupla com o tamanho das entradas.
inp = tf.keras.Input((x_train.shape[1], )) # x_train.shape[1] é o numero de colunas do nosso dataset. (Ou seja, quantos X's dos slides teremos)

# CAMADA OCULTA
# Camada oculta com 50 neurônios e função de ativação 'relu'
# Activation function to use. If you don't specify anything, no activation is applied (ie. "linear" activation: a(x) = x).
# ReLU(x)=max(0,x), isso significa que para uma entrada x, a saída será x se x for maior que 0, caso contrário, a saída será 0.
hide = tf.keras.layers.Dense(100, activation='relu', input_shape=(x_train.shape[1],))(inp)
hide = tf.keras.layers.Dense(100, activation='relu', input_shape=(x_train.shape[1],))(hide)
# CAMADA DE SAÍDA
# Camada de saída com 1 neurônio e função de ativação 'sigmoid'
out = tf.keras.layers.Dense(1, activation='sigmoid')(hide)



# CRIAÇÃO DO MODELO
modelo = tf.keras.Model(inputs = inp, outputs = out)
# No keras, existem 2 objetos que permitem com que se crie uma rede neural, o Sequencial e o Model (redes mais complexas).

# Compilando o modelo
modelo.compile(optimizer='adam', loss='msle')

# Visualizando modelo
modelo.summary()

# Treinando o modelo
modelo.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, shuffle=True, batch_size=1, validation_split=0.1)
# Batch são os lotes em que os dados são divididos para que não seja calculado o gradiente de uma vez
#Epochs são as passadas sobre todos os dados
# Suffle é tornar as passadas aleatórias

C:\Users\USUARIO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "functional_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_9 (InputLayer)      │ (None, 3)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 100)            │           400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,601 (41.41 KB)

 Trainable params: 10,601 (41.41 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.1239 - val_loss: 0.1154
Epoch 2/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1170 - val_loss: 0.1164
Epoch 3/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1149 - val_loss: 0.1171
Epoch 4/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1136 - val_loss: 0.1175
Epoch 5/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1128 - val_loss: 0.1180
Epoch 6/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1121 - val_loss: 0.1180
Epoch 7/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1116 - val_loss: 0.1183
Epoch 8/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1111 - val_loss: 0.1181
Epoch 9/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1108 - val_loss: 0.1185
Epoch 10/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1104 - val_loss: 0.1184
Epoch 11/100
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1102 - val_loss: 0.1186
Epoch 12/100
140/140 ━━━━━━━━━━━━━━━━━━━━

In [138]:
# Como queremos prever um valor binário, então estamos tratando com um problema de classificação binária. Nesse caso, usamos as métricas de avaliação da classificação
# para definir que meu modelo de redes neurais está adequado ou não.

# Agora, faça previsões nos dados de teste
y_pred = modelo.predict(x_test)

# As previsões são probabilidades, então você pode querer convertê-las em valores binários
# Por exemplo, se a probabilidade for maior que 0.5, você pode considerar como classe positiva (1)
# Caso contrário, como classe negativa (0)
y_pred_binary = (y_pred > 0.5).astype(int)

# MÉTRICAS DE CLASSIFICAÇÃO

from sklearn.metrics import accuracy_score, precision_score, recall_score

accuracy = accuracy_score(y_test, y_pred_binary)
precision = precision_score(y_test, y_pred_binary)
recall = recall_score(y_test, y_pred_binary)

print(f'Acurácia: {accuracy}')
print(f'Precisão: {precision}')
print(f'Revocação: {recall}')

1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/stepWARNING:tensorflow:6 out of the last 6 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x0000017D84903380> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
Acurácia: 0.6333333333333333
Precisão: 0.6363636363636364
Revocação: 0.5
